In [2]:
# the standalone script to identify and classfy_nodel from Ningbo Data
# full_prep
from preprocessing import full_prep
from config_submit import config as config_submit

import torch
from torch.nn import DataParallel
from torch.backends import cudnn
from torch.utils.data import DataLoader
from torch import optim
from torch.autograd import Variable
from layers import acc

from data_detector import DataBowl3Detector,collate
from data_classifier import DataBowl3Classifier

from utils import *
from split_combine import SplitComb
from test_detect import test_detect
from importlib import import_module
import pandas
import numpy as np

datapath = config_submit['datapath']
prep_result_path = config_submit['preprocess_result_path']
skip_prep = True
skip_detect = False

if not skip_prep:
    print('It will take ups to 2min to do the preprocess, you will be able to skip it in a second run')
    testsplit, sliceim, extendbox = full_prep(datapath,prep_result_path,
                          n_worker = config_submit['n_worker_preprocessing'],
                          use_existing=config_submit['use_exsiting_preprocessing'])
    print('region of interest shape:',sliceim.shape)
    print('extendbox shape defined:',extendbox)
else:
    testsplit = os.listdir(datapath)
print('Loading QED nodule detection Master')
nodmodel = import_module(config_submit['detector_model'].split('.py')[0])
config1, nod_net, loss, get_pbb = nodmodel.get_model()
checkpoint = torch.load(config_submit['detector_param'])
nod_net.load_state_dict(checkpoint['state_dict'])

#torch.cuda.set_device(0)
#nod_net = nod_net.cuda()
#cudnn.benchmark = True
# Try working with CPU

nod_net = DataParallel(nod_net)

bbox_result_path = './bbox_result'
if not os.path.exists(bbox_result_path):
    os.mkdir(bbox_result_path)
testsplit = [f.split('_clean')[0] for f in os.listdir(prep_result_path) if '_clean' in f]
print testsplit
example = ['000014.mhd']
config1['crop_size'] = [32,32,32]
if not skip_detect:
    margin = 32
    sidelen = 144
    config1['datadir'] = prep_result_path
    split_comber = SplitComb(sidelen,config1['max_stride'],config1['stride'],margin,pad_value= config1['pad_value'])
    dataset = DataBowl3Detector(example,config1,phase='test',split_comber=split_comber)
    test_loader = DataLoader(dataset,batch_size = 1,
        shuffle = False,num_workers = 1,pin_memory=False,collate_fn =collate)
    print 'pk'
    #test_detect(test_loader, nod_net, get_pbb, bbox_result_path,config1,n_gpu=config_submit['n_gpu'])
    print 'success'

/Library/Python/2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Loading QED nodule detection Master
['8b9a2378b94d14526f266148c7991e78f20fd1f4', '06ea1ff317a8652bb74f6710efa3b758be96524e', '8a5a850d1f90a3b5e4cb6a3a137ac6be0c6768d9', 'ce1ff1a4-180d882f-5401a107-db5428b4-f58b0322', '6e0d4bef9a6a3c88d1c24cbb4e64d26526ad7cb1', 'ded4983b-85e8b6b0-9b9c0898-232b7d47-c2122a5d', 'c5aeb59f-9b52b436-4c12187a-fd90406d-fb55cc4b', '000014.mhd']
pk
success


In [11]:
import argparse
import os
import time
import numpy as np
from importlib import import_module
import shutil
from utils import *
import sys
from split_combine import SplitComb

import torch
from torch.nn import DataParallel
from torch.backends import cudnn
from torch.utils.data import DataLoader
from torch import optim
from torch.autograd import Variable
from layers import acc



net = nod_net
save_dir = bbox_result_path
config = config1
n_gpu = 1
start_time = time.time()
data_loader = test_loader
print data_loader
net.eval()
split_comber = data_loader.dataset.split_comber
print "---------"
for i_name, (data, target, coord, nzhw) in enumerate(data_loader):
    assert i_name ==0
    s = time.time()
    print "xxxxxxxxxx"
    target = [np.asarray(t, np.float32) for t in target]
    lbb = target[0]
    nzhw = nzhw[0]
    name = data_loader.dataset.filenames[i_name].split('-')[0].split('/')[-1]
    print name
    shortname = name.split('_clean')[0]
    print shortname
    data = data[0][0]
    #print data
    coord = coord[0][0]
    isfeat = False
    if 'output_feature' in config:
        if config['output_feature']:
            isfeat = True
    n_per_run = n_gpu
    print(data.size())
    splitlist = range(0,len(data)+1,n_gpu)
    if splitlist[-1]!=len(data):
        splitlist.append(len(data))
    print splitlist
    outputlist = []
    featurelist = []

    for i in range(len(splitlist)-1):
        input = Variable(data[splitlist[i]:splitlist[i+1]], volatile = True)#.cuda()
        print input
        inputcoord = Variable(coord[splitlist[i]:splitlist[i+1]], volatile = True)#.cuda()
        print inputcoord
        if isfeat:
            output,feature = net(input,inputcoord)
            featurelist.append(feature.data.cpu().numpy())
        else:
            output = net(input,inputcoord)
        outputlist.append(output.data.cpu().numpy())
    output = np.concatenate(outputlist,0)
    output = split_comber.combine(output,nzhw=nzhw)
    print "---------"
#     #if isfeat:
#     #    feature = np.concatenate(featurelist,0).transpose([0,2,3,4,1])[:,:,:,:,:,np.newaxis]
#     #    feature = split_comber.combine(feature,sidelen)[...,0]

#     thresh = -3
#     print output.shape
#     pbb,mask = get_pbb(output,thresh,ismask=True)
#     #if isfeat:
#     #    feature_selected = feature[mask[0],mask[1],mask[2]]
#     #    np.save(os.path.join(save_dir, shortname+'_feature.npy'), feature_selected)
#         #tp,fp,fn,_ = acc(pbb,lbb,0,0.1,0.1)
#         #print([len(tp),len(fp),len(fn)])
#     print([i_name,shortname])
#     e = time.time()
#     print pbb   
#     np.save(os.path.join(save_dir, shortname+'_pbb.npy'), pbb)
#     np.save(os.path.join(save_dir, shortname+'_lbb.npy'), lbb)
# end_time = time.time()


# print('elapsed time is %3.2f seconds' % (end_time - start_time))
# print
# print

---------
xxxxxxxxxx
000014.mhd_clean.npy
000014.mhd
torch.Size([12, 1, 208, 208, 208])
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


/Users/wushangzhen/Library/Python/2.7/lib/python/site-packages/ipykernel_launcher.py:59: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


tensor([[[[[ 0.3281,  0.3281,  0.3281,  ...,  0.3281,  0.3281,
             0.3281],
           [ 0.3281,  0.3281,  0.3281,  ...,  0.3281,  0.3281,
             0.3281],
           [ 0.3281,  0.3281,  0.3281,  ...,  0.3281,  0.3281,
             0.3281],
           ...,
           [ 0.3281,  0.3281,  0.3281,  ...,  0.3281,  0.3281,
             0.3281],
           [ 0.3281,  0.3281,  0.3281,  ...,  0.3281,  0.3281,
             0.3281],
           [ 0.3281,  0.3281,  0.3281,  ...,  0.3281,  0.3281,
             0.3281]],

          [[ 0.3281,  0.3281,  0.3281,  ...,  0.3281,  0.3281,
             0.3281],
           [ 0.3281,  0.3281,  0.3281,  ...,  0.3281,  0.3281,
             0.3281],
           [ 0.3281,  0.3281,  0.3281,  ...,  0.3281,  0.3281,
             0.3281],
           ...,
           [ 0.3281,  0.3281,  0.3281,  ...,  0.3281,  0.3281,
             0.3281],
           [ 0.3281,  0.3281,  0.3281,  ...,  0.3281,  0.3281,
             0.3281],
           [ 0.3281,  0.3281,  

/Users/wushangzhen/Library/Python/2.7/lib/python/site-packages/ipykernel_launcher.py:61: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


tensor([[[[[ 0.3281,  0.3281,  0.3281,  ...,  0.3281,  0.3281,
             0.3281],
           [ 0.3281,  0.3281,  0.3281,  ...,  0.3281,  0.3281,
             0.3281],
           [ 0.3281,  0.3281,  0.3281,  ...,  0.3281,  0.3281,
             0.3281],
           ...,
           [ 0.3281,  0.3281,  0.3281,  ...,  0.3281,  0.3281,
             0.3281],
           [ 0.3281,  0.3281,  0.3281,  ...,  0.3281,  0.3281,
             0.3281],
           [ 0.3281,  0.3281,  0.3281,  ...,  0.3281,  0.3281,
             0.3281]],

          [[ 0.3281,  0.3281,  0.3281,  ...,  0.3281,  0.3281,
             0.3281],
           [ 0.3281,  0.3281,  0.3281,  ...,  0.3281,  0.3281,
             0.3281],
           [ 0.3281,  0.3281,  0.3281,  ...,  0.3281,  0.3281,
             0.3281],
           ...,
           [ 0.3281,  0.3281,  0.3281,  ...,  0.3281,  0.3281,
             0.3281],
           [ 0.3281,  0.3281,  0.3281,  ...,  0.3281,  0.3281,
             0.3281],
           [ 0.3281,  0.3281,  

...

tensor([[[[[ 0.3281,  0.3281,  0.3281,  ...,  0.3281,  0.3281,
             0.3281],
           [ 0.3281,  0.3281,  0.3281,  ...,  0.3281,  0.3281,
             0.3281],
           [ 0.3281,  0.3281,  0.3281,  ...,  0.3281,  0.3281,
             0.3281],
           ...,
           [-0.6562, -0.6562, -0.6328,  ...,  0.3281,  0.3281,
             0.3281],
           [-0.5781, -0.6406, -0.6875,  ...,  0.3281,  0.3281,
             0.3281],
           [-0.4297, -0.6797, -0.7422,  ...,  0.3281,  0.3281,
             0.3281]],

          [[ 0.3281,  0.3281,  0.3281,  ...,  0.3281,  0.3281,
             0.3281],
           [ 0.3281,  0.3281,  0.3281,  ...,  0.3281,  0.3281,
             0.3281],
           [ 0.3281,  0.3281,  0.3281,  ...,  0.3281,  0.3281,
             0.3281],
           ...,
           [-0.7422, -0.7344, -0.7031,  ...,  0.3281,  0.3281,
             0.3281],
           [-0.5000, -0.4844, -0.5469,  ...,  0.3281,  0.3281,
             0.3281],
           [-0.6562, -0.6016, -

tensor([[[[[ 0.3281,  0.3281,  0.3281,  ...,  0.3281,  0.3281,
             0.3281],
           [ 0.3281,  0.3281,  0.3281,  ...,  0.3281,  0.3281,
             0.3281],
           [ 0.3281,  0.3281,  0.3281,  ...,  0.3281,  0.3281,
             0.3281],
           ...,
           [-0.6094, -0.6953, -0.7188,  ...,  0.3281,  0.3281,
             0.3281],
           [-0.6875, -0.6641, -0.6797,  ...,  0.3281,  0.3281,
             0.3281],
           [-0.6875, -0.7031, -0.7734,  ...,  0.3281,  0.3281,
             0.3281]],

          [[ 0.3281,  0.3281,  0.3281,  ...,  0.3281,  0.3281,
             0.3281],
           [ 0.3281,  0.3281,  0.3281,  ...,  0.3281,  0.3281,
             0.3281],
           [ 0.3281,  0.3281,  0.3281,  ...,  0.3281,  0.3281,
             0.3281],
           ...,
           [-0.6094, -0.5000, -0.5938,  ...,  0.3281,  0.3281,
             0.3281],
           [-0.7266, -0.7266, -0.7031,  ...,  0.3281,  0.3281,
             0.3281],
           [-0.3828, -0.5078, -

tensor([[[[[ 0.3281,  0.3281,  0.3281,  ...,  0.1719,  0.4375,
             0.2578],
           [ 0.3281,  0.3281,  0.3281,  ...,  0.2969,  0.3281,
             0.2344],
           [ 0.3281,  0.3281,  0.3281,  ...,  0.2344,  0.3672,
             0.3672],
           ...,
           [ 0.3281,  0.3281,  0.3281,  ...,  0.3281,  0.3281,
             0.3281],
           [ 0.3281,  0.3281,  0.3281,  ...,  0.3281,  0.3281,
             0.3281],
           [ 0.3281,  0.3281,  0.3281,  ...,  0.3281,  0.3281,
             0.3281]],

          [[ 0.3281,  0.3281,  0.3281,  ...,  0.1406,  0.1484,
             0.2266],
           [ 0.3281,  0.3281,  0.3281,  ...,  0.3516,  0.3203,
             0.3047],
           [ 0.3281,  0.3281,  0.3281,  ...,  0.2031,  0.3281,
             0.2266],
           ...,
           [ 0.3281,  0.3281,  0.3281,  ...,  0.3281,  0.3281,
             0.3281],
           [ 0.3281,  0.3281,  0.3281,  ...,  0.3281,  0.3281,
             0.3281],
           [ 0.3281,  0.3281,  

tensor([[[[[-0.7344, -0.7734, -0.6797,  ..., -0.6328, -0.5625,
            -0.6484],
           [-0.7031, -0.6641, -0.6016,  ..., -0.6406, -0.6797,
            -0.6172],
           [-0.7109, -0.7578, -0.6953,  ..., -0.6797, -0.6484,
            -0.6562],
           ...,
           [ 0.3281,  0.3281,  0.3281,  ...,  0.3281,  0.3281,
             0.3281],
           [ 0.3281,  0.3281,  0.3281,  ...,  0.3281,  0.3281,
             0.3281],
           [ 0.3281,  0.3281,  0.3281,  ...,  0.3281,  0.3281,
             0.3281]],

          [[-0.6641, -0.6562,  0.2266,  ..., -0.6641, -0.7656,
            -0.7188],
           [-0.7734, -0.7812, -0.0859,  ..., -0.7188, -0.5312,
            -0.5781],
           [-0.6875, -0.7656, -0.2109,  ..., -0.7266, -0.5625,
            -0.6641],
           ...,
           [ 0.3281,  0.3281,  0.3281,  ...,  0.3281,  0.3281,
             0.3281],
           [ 0.3281,  0.3281,  0.3281,  ...,  0.3281,  0.3281,
             0.3281],
           [ 0.3281,  0.3281,  

tensor([[[[[-0.7422, -0.7031, -0.6562,  ...,  0.3281,  0.3281,
             0.3281],
           [-0.6250, -0.6172, -0.6797,  ...,  0.3281,  0.3281,
             0.3281],
           [-0.7109, -0.6953, -0.6875,  ...,  0.3281,  0.3281,
             0.3281],
           ...,
           [ 0.3281,  0.3281,  0.3281,  ...,  0.3281,  0.3281,
             0.3281],
           [ 0.3281,  0.3281,  0.3281,  ...,  0.3281,  0.3281,
             0.3281],
           [ 0.3281,  0.3281,  0.3281,  ...,  0.3281,  0.3281,
             0.3281]],

          [[-0.4531, -0.5156, -0.6406,  ...,  0.3281,  0.3281,
             0.3281],
           [-0.7188, -0.7734, -0.7734,  ...,  0.3281,  0.3281,
             0.3281],
           [-0.5547, -0.6953, -0.7344,  ...,  0.3281,  0.3281,
             0.3281],
           ...,
           [ 0.3281,  0.3281,  0.3281,  ...,  0.3281,  0.3281,
             0.3281],
           [ 0.3281,  0.3281,  0.3281,  ...,  0.3281,  0.3281,
             0.3281],
           [ 0.3281,  0.3281,  

In [ ]:
%matplotlib inline